In [6]:
import json
import base64
import tensorflow as tf
import pandas as pd
from tensorflow.contrib.predictor import from_saved_model
import s3fs
import os
import boto3
s3_client = boto3.client('s3', region_name='us-east-2')

In [7]:
!saved_model_cli show --all --dir model #save model


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['predict_images']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['image_bytes'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: ParseExample/ParseExample:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['classes'] tensor_info:
        dtype: DT_STRING
        shape: (-1, 4)
        name: index_to_string_Lookup:0
    outputs['scores'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 4)
        name: TopKV2:0
  Method name is: tensorflow/serving/predict

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['image_bytes'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: ParseExample/ParseExample:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['classes'] tensor_info:
        dtype: DT_

In [8]:
model=from_saved_model("model")
fs = s3fs.S3FileSystem() #call the s3fs to operate the filesystem

INFO:tensorflow:Restoring parameters from model/variables/variables


In [9]:
image = fs.ls('s3:...') #should be changed depend on the folder path

In [11]:
files = image[1:]
files

['ingo-images/Fiona_Test/xdmj_cropped/cropped_xdmj1-019_A01-1_BRITE_200207_151809_1.jpg',
 'ingo-images/Fiona_Test/xdmj_cropped/cropped_xdmj1-019_A01-1_BRITE_200207_180003_1.jpg',
 'ingo-images/Fiona_Test/xdmj_cropped/cropped_xdmj1-019_A01-1_BRITE_200207_201522_1.jpg',
 'ingo-images/Fiona_Test/xdmj_cropped/cropped_xdmj1-019_A01-1_BRITE_200207_211425_1.jpg',
 'ingo-images/Fiona_Test/xdmj_cropped/cropped_xdmj1-019_A01-1_BRITE_200207_221330_1.jpg',
 'ingo-images/Fiona_Test/xdmj_cropped/cropped_xdmj1-019_A01-1_BRITE_200207_231246_1.jpg',
 'ingo-images/Fiona_Test/xdmj_cropped/cropped_xdmj1-019_A01-1_BRITE_200211_173843_1.jpg',
 'ingo-images/Fiona_Test/xdmj_cropped/cropped_xdmj1-019_A01-1_BRITE_200218_180617_1.jpg',
 'ingo-images/Fiona_Test/xdmj_cropped/cropped_xdmj1-019_A01-1_BRITE_200227_173837_1.jpg',
 'ingo-images/Fiona_Test/xdmj_cropped/cropped_xdmj1-019_A01-1_BRITE_200307_180901_1.jpg',
 'ingo-images/Fiona_Test/xdmj_cropped/cropped_xdmj1-019_A01-1_BRITE_200414_192457_1.jpg',
 'ingo-ima

In [ ]:
for file in files:
    fp = fs.open('s3://'+file)
    data = fp.read()
    f_data = base64.b64encode(data)
    f_data = f_data.decode('utf-8')
    data={"image_bytes":{"b64":f_data}}
    beg = file.rfind("/")
    end = file.rfind(".")
    localfile = file[beg+1:end]+".json"
    with open(localfile,'w') as f:
        json.dump(data,f)
    response = s3_client.upload_file(localfile,'ingo-images','Fiona_Test/xdmj_json/'+localfile)#should be changed
    if os.path.exists(localfile):
        os.remove(localfile)
    
    #open a TEMPFILE
    #read the data into temp file
    #upload data to s3
    #closeTEMPFILE
    #f = open


    #with open(file[:76] + ".json","w") as f:
        #json.dump(data,f)

In [11]:

fp = fs.open('s3://...')
#image_path="unknown1.jpg"
data = fp.read()
#print(data)
f_data = base64.b64encode(data)
#print(f_data)
f_data = f_data.decode('utf-8')
#print(f_data)

data={"image_bytes":{"b64":f_data}}

print(data)

with open("batch_transform.json","w") as f:
    json.dump(data,f)

{'image_bytes': {'b64': '...'}}


In [25]:
!aws s3 cp batch_transform.json s3://ingo-images/Batch_test_json_1/batch_transform.json

upload: ./batch_transform.json to s3://ingo-images/Batch_test_json_1/batch_transform.json


In [4]:
!aws s3 cp crystal222.json s3://ingo-images/Batch_test_json_1/crystal222.json

upload: ./crystal222.json to s3://ingo-images/Batch_test_json_1/crystal222.json
